In [ ]:
import os, sys
import glob
import six.moves.cPickle as pkl
import numpy as np
import PIL.Image as Image
from skimage.transform import resize


def resize_mscoco():
    '''
    function used to create the dataset,
    Resize original MS_COCO Image into 64x64 images
    '''

    ### PATH need to be fixed
    data_path=r"C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting"
    save_dir = "/Tmp/64_64/train2014/"

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    preserve_ratio = True
    image_size = (64, 64)
    #crop_size = (32, 32)

    imgs = glob.glob(data_path+"/*.jpg")


    for i, img_path in enumerate(imgs):
        img = Image.open(img_path)
        print( i, len(imgs), img_path)

        if img.size[0] != image_size[0] or img.size[1] != image_size[1] :
            if not preserve_ratio:
                img = img.resize((image_size), Image.ANTIALIAS)
            else:
                ### Resize based on the smallest dimension
                scale = image_size[0] / float(np.min(img.size))
                new_size = (int(np.floor(scale * img.size[0]))+1, int(np.floor(scale * img.size[1])+1))
                img = img.resize((new_size), Image.ANTIALIAS)

                ### Crop the 64/64 center
                tocrop = np.array(img)
                center = (int(np.floor(tocrop.shape[0] / 2.)), int(np.floor(tocrop.shape[1] / 2.)))
                print (tocrop.shape, center, (center[0]-32,center[0]+32), (center[1]-32,center[1]+32))
                if len(tocrop.shape) == 3:
                    tocrop = tocrop[center[0]-32:center[0]+32, center[1] - 32:center[1]+32, :]
                else:
                    tocrop = tocrop[center[0]-32:center[0]+32, center[1] - 32:center[1]+32]
                img = Image.fromarray(tocrop)

        img.save(save_dir + os.path.basename(img_path))




def show_examples(batch_idx, batch_size,
                  ### PATH need to be fixed
                  mscoco=r'C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting', split="train2014", caption_path="dict_key_imgID_value_caps_train_and_valid.pkl"):
    '''
    Show an example of how to read the dataset
    '''
    save_dir1=r"C:\Users\NAVPREET\AnacondaProjects\training_data"
    save_dir2=r"C:\Users\NAVPREET\AnacondaProjects\target"
    data_path = os.path.join(mscoco, split)
    caption_path = os.path.join(mscoco, caption_path )
    b_train=[]
    b_target=[]
    b_tr=[]
    b_tar=[]
    with open(caption_path, 'rb') as fd:
        caption_dict = pkl.load(fd)

    print (data_path + "/*.jpg")
    imgs = glob.glob(data_path + "/*.jpg")
    batch_imgs = imgs[batch_idx*batch_size:(batch_idx+1)*batch_size]

    for i, img_path in enumerate(batch_imgs):
        img = Image.open(img_path)
        img_array = np.array(img)

        cap_id = os.path.basename(img_path)[:]

        ### Get input/target from the images
        center = (int(np.floor(img_array.shape[0] / 2.)), int(np.floor(img_array.shape[1] / 2.)))
        if len(img_array.shape) == 3:
            input = np.copy(img_array)
            input[center[0]-16:center[0]+16, center[1]-16:center[1]+16, :] = 0
            target = img_array[center[0]-16:center[0]+16, center[1] - 16:center[1]+16, :]
        else:
            input = np.copy(img_array)
            input[center[0]-16:center[0]+16, center[1]-16:center[1]+16, :] = 0
            target = img_array[center[0]-16:center[0]+16, center[1] - 16:center[1]+16]

        b_train.append(np.array(input).transpose(2,0,1).reshape(3,64,64))
        b_target.append(np.array(target).transpose(2,0,1).reshape(3,32,32))
        b_train=np.concatenate(b_train, axis=0 )
        b_target=np.concatenate(b_target, axis=0 )
        b_train = np.reshape(b_train, (len(b_train), 3, 64, 64))
        b_target = np.reshape(b_target, (len(b_target), 3, 32, 32)) 
        #Image.fromarray(img_array).show()
        #img1=Image.fromarray(input)
        #img2=Image.fromarray(target)
        ##b_tr.append(img1)
        #b_tar.append(img2)
        #b_tr=Image.fromarray([b_train])
        #b_train.reshape[batch_size, 3, 64, 64]
        #b_target.reshape[batch_size, 3, 32, 32]
        return b_train, b_target
        #img1.save(save_dir1 + os.path.basename(img_path))
        #img2.save(save_dir2 + os.path.basename(img_path))
        #b_train=b_train.append(img1)
        #b_target=b_target.append(img2)
        #print( i, caption_dict[cap_id])

def show_examples1(batch_idx, batch_size,
                  ### PATH need to be fixed
                  mscoco=r'C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting', split="val2014", caption_path="dict_key_imgID_value_caps_train_and_valid.pkl"):
    '''
    Show an example of how to read the dataset
    '''
    save_dir1=r"C:\Users\NAVPREET\AnacondaProjects\validation_data"
    save_dir2=r"C:\Users\NAVPREET\AnacondaProjects\val_target"
    data_path = os.path.join(mscoco, split)
    caption_path = os.path.join(mscoco, caption_path )
    b_train=[]
    b_target=[]
    b_tar=[]
    b_tr=[]
    with open(caption_path, 'rb') as fd:
        caption_dict = pkl.load(fd)

    print (data_path + "/*.jpg")
    imgs = glob.glob(data_path + "/*.jpg")
    batch_imgs = imgs[batch_idx*batch_size:(batch_idx+1)*batch_size]

    for i, img_path in enumerate(batch_imgs):
        img = Image.open(img_path)
        img_array = np.array(img)

        cap_id = os.path.basename(img_path)[:]

        ### Get input/target from the images
        center = (int(np.floor(img_array.shape[0] / 2.)), int(np.floor(img_array.shape[1] / 2.)))
        if len(img_array.shape) == 3:
            input = np.copy(img_array)
            input[center[0]-16:center[0]+16, center[1]-16:center[1]+16, :] = 0
            target = img_array[center[0]-16:center[0]+16, center[1] - 16:center[1]+16, :]
        else:
            input = np.copy(img_array)
            input[center[0]-16:center[0]+16, center[1]-16:center[1]+16, :] = 0
            target = img_array[center[0]-16:center[0]+16, center[1] - 16:center[1]+16]

        b_train.append(np.array(input).transpose(2,0,1).reshape(3,64,64))
        b_target.append(np.array(target).transpose(2,0,1).reshape(3,32,32))
        b_train=np.concatenate(b_train, axis=0 )
        b_target=np.concatenate(b_target, axis=0 )
        b_train = np.reshape(b_train, (batch_size, 3, 64, 64))
        b_target = np.reshape(b_target, (batch_size, 3, 32, 32)) 
        #y_train=np.concatenate(y_tr, axis=0 )
        #y_train=np.concatenate(y_tr, axis=0 )
        #Image.fromarray(img_array).show()
        #img1=Image.fromarray(input)
        #img2=Image.fromarray(target)
        #img1.save(save_dir1 + os.path.basename(img_path))
        #img2.save(save_dir2 + os.path.basename(img_path))
        #b_tr.append(img1)
        #b_tar.append(img2)
        #b_tr=Image.fromarray([b_train])
        return b_train, b_target
        #print( i, caption_dict[cap_id])



if __name__ == '__main__':
    #resize_mscoco()
    #X_tr, y_tr=show_examples(1,50)
    #X_train=np.concatenate(X_tr, axis=0 )
    #y_train=np.concatenate(y_tr, axis=0 )
    #print(len(X_train))
    #print(len(y_train))
    #X_v,y_v=show_examples1(5, 10)
    #X_val=np.concatenate(X_v, axis=0 )
    #y_val=np.concatenate(y_v, axis=0 )
    #print(len(X_val))
    #print(len(y_val))
    #X_train= np.Reshape([(X_train, (len(X_train), 3, 64, 64))
    #X_train= np.Reshape([(y_train, (len(y_train), 3, 32, 32))
    #X_val= np.Reshape([(X_val, (len(X_val), 3, 64, 64))
    #y_val= np.Reshape([(y_val, (len(y_val), 3, 32, 32))
    